In [1]:
#TO DO

# - Create correct directories in get_y
# - Do I want to make the padding value 0 so i ignore all rows with 0??
# why do I get loss nan??
# I have nans in my X_train /X_test
# should i use a different optimizer for my compiler?? maybe rmsprop

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import pandas as pd
import os
import csv
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
ROOT_DATA_DIR = '/content/drive/MyDrive/Squat_Pro/CSV_features'
#ROOT_DATA_DIR = '../raw_data/CSV_Features'
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]

def get_y(data_path, class_list):
    
    data_dict = {"csv_paths":[], "csv_files":[], "labels":[]}
    
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_csvs = [l for l in os.listdir(dir_path)]
        print(f"Found {len(list_csvs)} CSVs in {dir_path} Directory")
        data_dict["csv_files"].extend(list_csvs)
        data_dict["labels"].extend([i for k in range(len(list_csvs))])
        data_dict["csv_paths"].extend([dir_path for _ in range(len(list_csvs))])
    
    total_csvs = len(data_dict["csv_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of csvs : {total_csvs}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST))


test_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST))  

Found 40 CSVs in /content/drive/MyDrive/Squat_Pro/CSV_features/Train/Bad Directory
Found 40 CSVs in /content/drive/MyDrive/Squat_Pro/CSV_features/Train/Good Directory
Number of csvs : 80
Number of Labels : 80
Found 10 CSVs in /content/drive/MyDrive/Squat_Pro/CSV_features/Test/Bad Directory
Found 10 CSVs in /content/drive/MyDrive/Squat_Pro/CSV_features/Test/Good Directory
Number of csvs : 20
Number of Labels : 20


In [5]:
train_data.head()

,csv_paths,csv_files,labels
0,/content/drive/MyDrive/Squat_Pro/CSV_features/...,76.csv,0
1,/content/drive/MyDrive/Squat_Pro/CSV_features/...,77.csv,0
2,/content/drive/MyDrive/Squat_Pro/CSV_features/...,49.csv,0
3,/content/drive/MyDrive/Squat_Pro/CSV_features/...,41.csv,0
4,/content/drive/MyDrive/Squat_Pro/CSV_features/...,52.csv,0


In [6]:
train_data['csv_paths'][0]

'/content/drive/MyDrive/Squat_Pro/CSV_features/Train/Bad'

In [7]:
train_data.tail()

,csv_paths,csv_files,labels
75,/content/drive/MyDrive/Squat_Pro/CSV_features/...,39.csv,1
76,/content/drive/MyDrive/Squat_Pro/CSV_features/...,22.csv,1
77,/content/drive/MyDrive/Squat_Pro/CSV_features/...,4.csv,1
78,/content/drive/MyDrive/Squat_Pro/CSV_features/...,33.csv,1
79,/content/drive/MyDrive/Squat_Pro/CSV_features/...,14.csv,1


In [8]:
y_train = train_data['labels']

In [9]:
y_train

0     0
1     0
2     0
3     0
4     0
     ..
75    1
76    1
77    1
78    1
79    1
Name: labels, Length: 80, dtype: int64

In [10]:
y_test = test_data['labels']

In [11]:
df = pd.read_csv('/content/drive/MyDrive/Squat_Pro/CSV_features/Train/Bad/76.csv')
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,feature_1
0,0,472.5,1290.0,472.5,1080.0,472.500,900.0,0.000,0.0,0.000,0.0,0.0,0.0,506.250,900.0,0.0,0.0,523.125,660.0,540.000,450.0,556.875,930.0,523.125,810.0,506.25,660.0,0.0,0.0,590.625,840.0,0.0,0.0,180.000000
1,1,472.5,1290.0,472.5,1110.0,489.375,900.0,0.000,0.0,455.625,1140.0,0.0,0.0,489.375,900.0,0.0,0.0,523.125,660.0,540.000,450.0,556.875,930.0,523.125,810.0,506.25,660.0,0.0,0.0,0.000,0.0,0.0,0.0,175.405747
2,2,472.5,1290.0,472.5,1110.0,489.375,900.0,523.125,930.0,0.000,0.0,0.0,0.0,506.250,900.0,0.0,0.0,523.125,660.0,540.000,450.0,573.750,930.0,523.125,810.0,506.25,660.0,0.0,0.0,0.000,0.0,0.0,0.0,175.405747
3,3,472.5,1290.0,472.5,1110.0,489.375,900.0,523.125,930.0,455.625,1140.0,0.0,0.0,506.250,900.0,0.0,0.0,523.125,660.0,523.125,450.0,573.750,930.0,523.125,810.0,506.25,660.0,0.0,0.0,0.000,0.0,0.0,0.0,175.405747
4,4,472.5,1290.0,472.5,1110.0,489.375,900.0,506.250,930.0,472.500,1140.0,0.0,0.0,489.375,900.0,540.0,660.0,523.125,660.0,523.125,420.0,573.750,930.0,523.125,810.0,506.25,660.0,0.0,0.0,0.000,0.0,0.0,0.0,175.405747


In [12]:
def csv_to_list_of_lists(csv_path):
    df = pd.read_csv(csv_path)
    df.fillna(0, inplace=True)
    list_of_lists = df.feature_1.values.tolist()
    return list_of_lists

In [13]:
def get_x(dataframe):
  list_of_csv_files = [os.path.join(row['csv_paths'], row["csv_files"]) for index, row in dataframe.iterrows()]
  X_list = [csv_to_list_of_lists(f) for f in list_of_csv_files]
  return X_list

In [14]:
#X_train is a list of lists of different list lengths
X_train = get_x(train_data)

In [15]:
#normalizing the X_train
X_train = [[float(j)/180 for j in i] for i in X_train] 

In [16]:
X_train

[[1.0,
  0.9744763703818535,
  0.9744763703818535,
  0.9744763703818535,
  0.9744763703818535,
  0.9776556101276666,
  0.046768952390204246,
  0.046768952390204246,
  0.046768952390204246,
  0.044328285439493284,
  0.046768952390204246,
  0.9053419888533671,
  0.9053419888533671,
  0.34147146334059847,
  0.34132290842151697,
  0.129070566157862,
  0.34147146334059847,
  0.044328285439493284,
  0.9776556101276666,
  0.9776556101276666,
  0.029457970501539435,
  0.029457970501539435,
  0.9702454164285824,
  0.1744870455199965,
  0.34147146334059847,
  0.9744763703818535,
  0.9489527407637072,
  0.047683269002350405,
  0.9419956838485616,
  0.9447217868104357,
  0.8902787124531454,
  0.8902787124531454,
  0.8374311004069589,
  0.03733732313072343,
  0.6491711849664141,
  0.03733732313072343,
  0.008852041685682845,
  6.707879276254073e-09,
  6.707879276254073e-09,
  6.707879276254073e-09,
  6.707879276254073e-09,
  4.743186923619966e-09,
  4.743186923619966e-09,
  0.0,
  0.0,
  0.0,
  0.0

In [17]:
#pad X_train so all arrays are of the same shape
X_train_pad = pad_sequences(X_train, dtype='float32',padding='post',value=-1000)
X_train_pad.shape

(80, 176)

In [18]:
#input data must be of shape(number of sequences, number of observations per sequence, observation size)
#add a last dimension of 1
X_train_pad = np.expand_dims(X_train_pad, 2)
X_train_pad.shape

(80, 176, 1)

In [19]:
#X_test is a list of lists of different list lengths
X_test = get_x(test_data)

In [20]:
#normalizing the X_test
X_test = [[float(j)/180 for j in i] for i in X_test] 

In [21]:
#pad X_test so all arrays are of the same shape
X_test_pad = pad_sequences(X_test, dtype='float32',padding='post',value=-1000)

#input data must be of shape(number of sequences, number of observations per sequence, observation size)
#add a last dimension of 1
X_test_pad = np.expand_dims(X_test_pad, 2)

In [22]:
#LSTM RNN Model
model_LSTM = Sequential()

##change masking layer to padded amount
model_LSTM.add(layers.Masking(mask_value=-1000))

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(20, activation='tanh'))  
model_LSTM.add(layers.Dense(10, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))

model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#More Complex LSTM RNN Model
model_LSTM = Sequential()

##change masking layer to padded amount
model_LSTM.add(layers.Masking(mask_value=-1.))

##Hogan's first layer (do i need to add an input dimension??)
model.add(layers.SimpleRNN(500, input_dim=1, activation='relu'))

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(units=500, activation='tanh'))  
model_LSTM.add(layers.Dense(250, activation='relu'))
model_LSTM.add(layers.Dense(125, activation='relu'))
model_LSTM.add(layers.Dense(75, activation='relu'))
model_LSTM.add(layers.Dense(25, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
#Fitting the model to the train set
es = EarlyStopping(patience=2)

model_LSTM.fit(X_train_pad, y_train, 
              epochs=10, 
              batch_size=32, 
              verbose=1, 
              callbacks = [es],
              validation_split=0.2)

Epoch 1/10
2/2 [==============================] - 6s 3s/step - loss: 0.7054 - accuracy: 0.3958 - val_loss: 0.6767 - val_accuracy: 0.5625
Epoch 2/10
2/2 [==============================] - 0s 123ms/step - loss: 0.6986 - accuracy: 0.5104 - val_loss: 0.6884 - val_accuracy: 0.3125
Epoch 3/10
2/2 [==============================] - 0s 125ms/step - loss: 0.6959 - accuracy: 0.5521 - val_loss: 0.7013 - val_accuracy: 0.1875


In [24]:
#Evaluate the model on the test set
model_LSTM.evaluate(X_test_pad, y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.6880 - accuracy: 0.5500


[0.6880203485488892, 0.550000011920929]